In [ ]:
pip install pandas

In [ ]:
pip install tensorflow

In [ ]:
pip install python-dotenv

In [ ]:
import pandas
from dotenv import load_dotenv
import os
import requests
import sklearn
from sklearn.model_selection import train_test_split
import tensorflow

In [ ]:
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [ ]:
tensorflow.config.list_physical_devices('GPU')

In [ ]:
farfetchData=pandas.read_csv("D:/farfetchdataset/current_farfetch_listings.csv")

In [ ]:
farfetchData

In [ ]:
load_dotenv()

In [ ]:
CURRENCY_API_KEY=os.getenv("CURRENCY_API_KEY")

In [ ]:
def getExchangeRates(currency):
    url='https://v6.exchangerate-api.com/v6/'+CURRENCY_API_KEY+'/latest/'+currency
    response=requests.get(url)
    return response.json()

In [ ]:
sgd=getExchangeRates('SGD')

In [ ]:
sgd['conversion_rates']['USD']

In [ ]:
farfetchData['priceInfo.initialPrice']=farfetchData['priceInfo.initialPrice'].map(lambda x: x*sgd['conversion_rates']['USD'])

In [ ]:
farfetchData['priceInfo.finalPrice']=farfetchData['priceInfo.finalPrice'].map(lambda x: x*sgd['conversion_rates']['USD'])

In [ ]:
farfetchData

In [ ]:
def roundPrices(price):
    if price<100:
        return round(price,-1)
    else: return round(price,-2)

In [ ]:
farfetchData['priceInfo.initialPrice']=farfetchData['priceInfo.initialPrice'].map(roundPrices)

In [ ]:
farfetchData

In [ ]:
farfetchData['priceInfo.finalPrice']=farfetchData['priceInfo.finalPrice'].map(roundPrices)

In [ ]:
imagesSource="D:/farfetchdataset/cutout-img/cutout/"

In [ ]:
images=pandas.concat([farfetchData['images.cutOut'],farfetchData['priceInfo.finalPrice']],axis=1)

In [ ]:
images

In [ ]:
sampledImages=images.sample(frac=1.0,random_state=1).reset_index(drop=True)

In [ ]:
sampledImages

In [ ]:
sampledImages['imagePath']=sampledImages['images.cutOut'].map(lambda x: imagesSource+x.split('/')[-1])

In [ ]:
sampledImages['imagePath'][0]

In [ ]:
sampledImages.to_csv('sampledImages.csv')

In [ ]:
train,test=train_test_split(sampledImages,train_size=.7,shuffle=True,random_state=1)

In [ ]:
train

In [ ]:
trainer=tensorflow.keras.preprocessing.image.ImageDataGenerator(rescale=1./255,validation_split=.2)
tester=tensorflow.keras.preprocessing.image.ImageDataGenerator(rescale=1./255,)

In [ ]:
trainImages=trainer.flow_from_dataframe(dataframe=train,
                            x_col='imagePath',
                            y_col='priceInfo.finalPrice',
                            target_size=(120,120),
                            color_mode='rgb',
                           class_mode='raw',
                           batch_size=32,
                           shuffle=True,
                           seed=42,
                           subset='training')

valImages=trainer.flow_from_dataframe(dataframe=train,
                            x_col='imagePath',
                            y_col='priceInfo.finalPrice',
                            target_size=(120,120),
                            color_mode='rgb',
                           class_mode='raw',
                           batch_size=32,
                           shuffle=True,
                           seed=42,
                           subset='validation')

testImages=tester.flow_from_dataframe(dataframe=test,
                            x_col='imagePath',
                            y_col='priceInfo.finalPrice',
                            target_size=(120,120),
                            color_mode='rgb',
                           class_mode='raw',
                           batch_size=32,
                           shuffle=False)

In [ ]:
print("Num GPUs Available: ", len(tensorflow.config.list_physical_devices('GPU')))

In [ ]:
input=tensorflow.keras.Input(shape=(120,120,3))
cnn=tensorflow.keras.layers.Conv2D(filters=16,kernel_size=(3,3),activation='relu')(input)
cnn=tensorflow.keras.layers.MaxPool2D()(cnn)
cnn=tensorflow.keras.layers.Conv2D(filters=32,kernel_size=(3,3),activation='relu')(cnn)
cnn=tensorflow.keras.layers.MaxPool2D()(cnn)
cnn=tensorflow.keras.layers.GlobalAveragePooling2D()(cnn)
cnn=tensorflow.keras.layers.Dense(64,activation='relu')(cnn)
output=tensorflow.keras.layers.Dense(1,activation='linear')(cnn)
model=tensorflow.keras.Model(inputs=input,outputs=output)

model.compile(optimizer='adam',loss='mse')

history=model.fit(trainImages,validation_data=valImages,
                  epochs=100,
                  callbacks=[tensorflow.keras.callbacks.EarlyStopping(monitor='val_loss',patience=5)])

In [ ]:
pred=numpy.squeeze(model.predict(testImages))

In [ ]:
rootMeanSq=numpy.sqrt(model.evaluate(testImages))
print(rootMeanSq)